In [7]:
import pandas as pd
import csv

height = int(input("Input your height: "))
weight = int(input("Input your weight: "))

bmi = weight / pow(height / 100, 2)
print("Your BMI:", bmi)

# meals = pd.read_csv('meals_loseweight.csv')
meals = pd.read_csv('meals_loseweight.csv', encoding='latin-1')


print(meals)
selection = int(input("Please input the mealId of the meal that you would like to order: "))
if meals.loc[0][1] == "Chicken and Cayenne Pepper":
    meal_ordered = meals.loc[selection - 1][1]
else:
    meal_ordered = meals.loc[selection - 21][1]
print("You have ordered", meal_ordered)
print("Thank you for your order")
meal_ordered_rate = int(input("What would you rate this meal? [1-5]: "))

ratings = pd.read_csv('rating.csv', encoding='latin-1')
ratings = pd.merge(meals, ratings).drop(['calories'], axis=1)
ratings.head()

user_ratings = ratings.pivot_table(index=['userId'], columns=['meal'], values='rating')
user_ratings.head()

user_ratings = user_ratings.fillna(0)
user_ratings.head()

item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50)

def get_similar_meals(meal_name, user_rating):
    similar_score = item_similarity_df[meal_name] * (user_rating - 2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

person = [(meal_ordered, meal_ordered_rate)]
similar_meals = pd.DataFrame()
for meal, rating in person:
    similar_meals = similar_meals.append(get_similar_meals(meal, rating), ignore_index=True)

print("Recommended meals based on your previous experience:")
similar_meals.head()
print(similar_meals.sum().sort_values(ascending=False))

with open('rating.csv') as file_obj:
    reader_obj = csv.reader(file_obj)
    for row in reader_obj:
        new_user = row[0]
new_user = int(new_user) + 1

data_to_append = [new_user, selection, meal_ordered_rate]
file = open('rating.csv', 'a', newline='\n', encoding='latin-1')
writer = csv.writer(file)
writer.writerow(data_to_append)
file.close()


Input your height: 180
Input your weight: 70
Your BMI: 21.604938271604937
    mealId                                               meal calories
0        1                         Chicken and Cayenne Pepper      150
1        2                                Oatmeal and Walnuts      166
2        3                            Bean and Vegetable Soup      193
3        4                            Salmon and Sweet Potato      227
4        5                          Mushrooms and Ground Beef      314
5        6              Fish, Whole-Wheat Tortilla, and Salsa      305
6        7                             Yogurt and Raspberries      112
7        8                            Citrus and herb chicken      347
8        9  Healthy harissa chicken, lentil and kale tray ...      318
9       10      Sweet potatoes with chicken and jalapeno slaw      312
10      11            Ginger pork stir-fry with vegie noodles      244
11      12               Sticky pork skewers with wombok slaw      442
12 

<ipython-input-7-b648c7eb82b6>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_meals = similar_meals.append(get_similar_meals(meal, rating), ignore_index=True)
